In [ ]:
import pandas as pd
# !pwd
dataset_path = "../data/raw/"
dataset_name = "ml-latest-small"
df_ratings = pd.read_csv(f"{dataset_path}{dataset_name}/ratings.csv")
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
# ratings_matrix = df_ratings.pivot(index='userId', columns='movieId', values='rating')
# ratings_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from scipy.sparse import coo_matrix
import pandas as pd

user_map = {u: i for i, u in enumerate(df_ratings['userId'].unique())}
movie_map = {m: i for i, m in enumerate(df_ratings['movieId'].unique())}
rows = df_ratings['userId'].map(user_map)
cols = df_ratings['movieId'].map(movie_map)
data = df_ratings['rating']
sparse_matrix = coo_matrix((data, (rows, cols)))

In [18]:
import implicit

# Convert to implicit format (prefer high rating = strong interaction)
sparse_matrix = sparse_matrix.tocsr()

# Initialize model
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=15)

# Train model
model.fit(sparse_matrix)

/home/ubuntu/movies/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/movies/.venv/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 15/15 [00:02<00:00,  6.36it/s]


In [46]:
user_id = 1

recommended = model.recommend(user_id, sparse_matrix[user_id], N=10)

df_movies = pd.read_csv(f"{dataset_path}ml-latest/movies.csv")
movies_dict = dict(zip(df_movies['movieId'], df_movies['title']))

for item_id, score in zip(recommended[0], recommended[1]):
    title = movies_dict.get(item_id, "Unknown Movie")
    print(f"{title}: {score:.3f}")

The Show (1995): 0.578
Doom Generation, The (1995): 0.555
Babe (1995): 0.544
Casino (1995): 0.528
Basketball Diaries, The (1995): 0.508
He Walked by Night (1948): 0.481
Money Train (1995): 0.467
Paradise Lost: The Child Murders at Robin Hood Hills (1996): 0.436
Nadja (1994): 0.432
Wend Kuuni (a.k.a. God's Gift) (1982): 0.430


In [59]:
user_ratings = df_ratings[df_ratings['userId'] == user_id]
user_ratings = user_ratings.merge(df_movies, on='movieId')
user_ratings = user_ratings[['title', 'rating']].sort_values(by='rating', ascending=False)
# print(user_ratings)
user_ratings

,title,rating
3,Seven (a.k.a. Se7en) (1995),5.0
4,"Usual Suspects, The (1995)",5.0
6,Bottle Rocket (1996),5.0
13,Dumb & Dumber (Dumb and Dumber) (1994),5.0
11,Billy Madison (1995),5.0
...,...,...
143,Toys (1992),2.0
152,Psycho (1998),2.0
148,I Still Know What You Did Last Summer (1998),2.0
170,"Mummy, The (1999)",2.0


In [53]:
user_ratings

,userId_x,movieId,rating_x,timestamp_x,userId_y,rating_y,timestamp_y
0,1,1,4.0,964982703,1,4.0,964982703
1,1,1,4.0,964982703,5,4.0,847434962
2,1,1,4.0,964982703,7,4.5,1106635946
3,1,1,4.0,964982703,15,2.5,1510577970
4,1,1,4.0,964982703,17,4.5,1305696483
...,...,...,...,...,...,...,...
16291,1,5060,5.0,964984002,577,4.0,945966132
16292,1,5060,5.0,964984002,597,5.0,941640278
16293,1,5060,5.0,964984002,600,5.0,1237850978
16294,1,5060,5.0,964984002,603,3.0,953926877


In [43]:
user_id

0

In [61]:
from implicit.evaluation import train_test_split

train, test = train_test_split(sparse_matrix, train_percentage=0.8)

model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(train)

100%|██████████| 15/15 [00:02<00:00,  6.65it/s]


In [ ]:
from implicit.evaluation import precision_at_k, mean_average_precision_at_k

def precision_at_k_manual(model, train_matrix, test_matrix, K=10):
    """
    Compute average Precision@K over all users who have at least one item in the test set.
    """
    num_users = train_matrix.shape[0]
    precisions = []

    for user_index in range(num_users):
        test_items = test_matrix[user_index].indices
        if len(test_items) == 0:
            continue

        seen_items = train_matrix[user_index].indices
        recommended = model.recommend(
            user_index,
            train_matrix[user_index],
            N=K + len(seen_items),
            filter_items=seen_items
        )

        # FIXED: unpack item IDs and take top K
        item_ids, _ = recommended
        top_k_items = item_ids[:K]

        hits = len(set(top_k_items) & set(test_items))
        precision = hits / K
        precisions.append(precision)

    return sum(precisions) / len(precisions) if precisions else 0.0


def mean_average_precision_at_k_manual(model, train_matrix, test_matrix, K=10):
    num_users = train_matrix.shape[0]
    average_precisions = []

    for user_index in range(num_users):
        test_items = test_matrix[user_index].indices
        if len(test_items) == 0:
            continue

        seen_items = train_matrix[user_index].indices
        recommended = model.recommend(
            user_index,
            train_matrix[user_index],
            N=K + len(seen_items),
            filter_items=seen_items
        )

        item_ids, _ = recommended
        top_k_items = item_ids[:K]

        hits = 0
        sum_precisions = 0.0

        for i, item_id in enumerate(top_k_items):
            if item_id in test_items:
                hits += 1
                precision_at_i = hits / (i + 1)
                sum_precisions += precision_at_i

        if hits > 0:
            average_precisions.append(sum_precisions / hits)
        else:
            average_precisions.append(0.0)

    return sum(average_precisions) / len(average_precisions) if average_precisions else 0.0


# These manual functions are quick fixes due to incompatibility issues with newer scipy versions

# Precision@10
# precision = precision_at_k(model, train, test, K=10)
precision = precision_at_k_manual(model, train, test, K=10)
print(f"Precision@10: {precision:.4f}")

# MAP@10
# mapk = mean_average_precision_at_k(model, train, test, K=10)
mapk = mean_average_precision_at_k_manual(model, train, test, K=10)
print(f"MAP@10: {mapk:.4f}")

Precision@10: 0.2685
MAP@10: 0.4642


How to interpret results:

1. Precision@K
✅ Definition:

Out of the top K recommendations, what fraction were actually relevant to the user?

More formally:

    Precision@K = (# of relevant items in top K) / K

🔍 Interpretation Example

If:

Precision@10 = 0.2685

Then it means:

    On average, 2.7 of the top 10 recommended movies were actually movies the user interacted with in the test set.

That's generally a decent result, especially for implicit data (where the user might like items they didn’t rate explicitly).
✅ Good Values:
Precision@10	Interpretation
0.0–0.1	Poor: recommendations are mostly irrelevant
0.1–0.3	Fair: average match rate
0.3–0.5	Good: high relevance in top results
> 0.5	Excellent (hard to achieve in real data)

    Real-world systems (Netflix, Amazon) usually hover around 0.3–0.4 for cold-start, higher when well-tuned.

🧠 2. MAP@K (Mean Average Precision)
✅ Definition:

MAP@K takes ranking into account, not just presence in the top-K. It rewards:

    Correct items that appear early in the top-K

    Penalizes relevant items that are ranked lower

🔍 Interpretation Example

If:

MAP@10 = 0.19

Then it means:

    On average, users are seeing relevant items fairly early in the top 10 — not just present, but ranked well.

✅ Good Values:
MAP@10	Interpretation
< 0.1	Poor: relevant items are not being ranked high
0.1–0.3	Typical for basic models
0.3–0.5	Strong performance
> 0.5	Very strong, likely with rich feature models
📈 Which Metric Should You Trust?
Use Case	Metric to Focus On
Recommend a few good items	Precision@K
Rank items very well	MAP@K or NDCG@K
Coverage / diversity focus	Add Recall@K or Hit Rate

In [69]:
import os
dataset_path = os.path.join('/', "../data/raw/movies.csv")
dataset_path

'/../data/raw/movies.csv'